# Redis Serialization

In this notebook, we will explain how to utilize serialization and deserialization function provided in sotopia, achieving the convertion between jsonline / CSV and redis database.

In [ ]:
import sys

sys.path.append("../")
import os

os.environ["REDIS_OM_URL"] = "redis://:QzmCUD3C3RdsR@localhost:6379"
from typing import get_args
from pydantic import ValidationError
from sotopia.database import (
    AgentProfile,
    EnvironmentProfile,
    RelationshipProfile,
    EpisodeLog,
)
from sotopia.database import (
    episodes_to_csv,
    episodes_to_jsonl,
    agentprofiles_to_csv,
    agentprofiles_to_jsonl,
    environmentprofiles_to_csv,
    environmentprofiles_to_jsonl,
    relationshipprofiles_to_csv,
    relationshipprofiles_to_jsonl,
    envagnetcombostorage_to_csv,
    envagnetcombostorage_to_jsonl,
    jsonl_to_episodes,
    jsonl_to_agentprofiles,
    jsonl_to_environmentprofiles,
    jsonl_to_relationshipprofiles,
    jsonl_to_envagnetcombostorage,
)
from sotopia.database.env_agent_combo_storage import EnvAgentComboStorage
from typing import Literal
from pprint import pprint

### Episodes Log Serialization

In [2]:
LLM_Name = Literal[
    "togethercomputer/llama-2-7b-chat",
    "togethercomputer/llama-2-70b-chat",
    "togethercomputer/mpt-30b-chat",
    "gpt-3.5-turbo",
    "text-davinci-003",
    "gpt-4",
    "gpt-4-turbo",
    "human",
    "redis",
]


def _is_valid_episode_log_pk(pk: str) -> bool:
    try:
        episode = EpisodeLog.get(pk=pk)
    except ValidationError:
        return False
    try:
        tag = episode.tag
        model_1, model_2, version = tag.split("_", maxsplit=2)
        if (
            model_1 in get_args(LLM_Name)
            and model_2 in get_args(LLM_Name)
            and version == "v0.0.1_clean"
        ):
            return True
        else:
            return False
    except (ValueError, AttributeError):
        # ValueError: tag has less than 3 parts
        # AttributeError: tag is None
        return False


episodes: list[EpisodeLog] = [
    EpisodeLog.get(pk=pk)
    for pk in filter(_is_valid_episode_log_pk, EpisodeLog.all_pks())
]
pprint(episodes[0])

EpisodeLog(pk='01H9C9BC1RBG8DJTVC426YJ0M0', environment='01H7VFHN7A1ZX5KSMT2YN9RXC4', agents=['01H5TNE5PQ00AJVSSVB9V2VA9K', '01H5TNE5P98J20AEW94XQ0KC35'], tag='togethercomputer/mpt-30b-chat_gpt-3.5-turbo_v0.0.1_clean', models=['gpt-4', 'gpt-3.5-turbo', 'togethercomputer/mpt-30b-chat'], messages=[[('Environment', 'Eli Dawson', "\nHere is the context of this interaction:\nScenario: Two friends at a garage sale\nParticipants: Eli Dawson and Ethan Smith\nEli Dawson's background: Eli Dawson is a 52-year-old male forensic psychiatrist. He/him pronouns. Eli Dawson, a forensic psychiatrist, is a history enthusiast and enjoys collecting vintage books. Personality and values description: Eli Dawson is expressive, assertive, and markedly erratic. Elliott values liberty and enjoyment. His decision-making is strategic, examining the wider context before making decisions. Eli's secrets: He secretly funds a college student\nEthan Smith's background: Ethan Smith is a 29-year-old male software develope

In [3]:
episodes_to_csv(episodes, "../data/sotopia_episodes_v1.csv")
episodes_to_jsonl(episodes, "../data/sotopia_episodes_v1.jsonl")
rebuild_episodes = jsonl_to_episodes("../data/sotopia_episodes_v1.jsonl")
pprint(rebuild_episodes[0])

EpisodeLog(pk='01H9C9BC1RBG8DJTVC426YJ0M0', environment='01H7VFHN7A1ZX5KSMT2YN9RXC4', agents=['01H5TNE5PQ00AJVSSVB9V2VA9K', '01H5TNE5P98J20AEW94XQ0KC35'], tag='togethercomputer/mpt-30b-chat_gpt-3.5-turbo_v0.0.1_clean', models=['gpt-4', 'gpt-3.5-turbo', 'togethercomputer/mpt-30b-chat'], messages=[[('Environment', 'Eli Dawson', "\nHere is the context of this interaction:\nScenario: Two friends at a garage sale\nParticipants: Eli Dawson and Ethan Smith\nEli Dawson's background: Eli Dawson is a 52-year-old male forensic psychiatrist. He/him pronouns. Eli Dawson, a forensic psychiatrist, is a history enthusiast and enjoys collecting vintage books. Personality and values description: Eli Dawson is expressive, assertive, and markedly erratic. Elliott values liberty and enjoyment. His decision-making is strategic, examining the wider context before making decisions. Eli's secrets: He secretly funds a college student\nEthan Smith's background: Ethan Smith is a 29-year-old male software develope

## Relationship Profile Serialization

In [4]:
pks = RelationshipProfile.all_pks()
pks = list(pks)
res = []
for pk in pks:
    try:
        res.append(RelationshipProfile.get(pk=pk))
    except:
        print("error")
        pass
pprint(res[0])

RelationshipProfile(pk='01H6HHQFQTWY57983SAKEK11KP', agent_1_id='01H5TNE5PBKCFDAK6293NKYJ4D', agent_2_id='01H5TNE5P98J20AEW94XQ0KC35', relationship=<RelationshipType.romantic_relationship: 4>, background_story="Ava Thompson and Ethan Smith are colleagues at a tech firm. Ava, an architect, frequently collaborates with Ethan, a software developer, on different projects. They know each other by name and occasionally engage in small talk. Despite their differing personalities and decision-making styles, they respect each other's professional skills and contributions.")


In [5]:
relationshipprofiles_to_csv(res, "../data/sotopia_relationshipprofiles_v1.csv")
relationshipprofiles_to_jsonl(res, "../data/sotopia_relationshipprofiles_v1.jsonl")
relationship_profiles = jsonl_to_relationshipprofiles(
    "../data/sotopia_relationshipprofiles_v1.jsonl"
)
pprint(relationship_profiles[0])

RelationshipProfile(pk='01H6HHQFQTWY57983SAKEK11KP', agent_1_id='01H5TNE5PBKCFDAK6293NKYJ4D', agent_2_id='01H5TNE5P98J20AEW94XQ0KC35', relationship=<RelationshipType.romantic_relationship: 4>, background_story="Ava Thompson and Ethan Smith are colleagues at a tech firm. Ava, an architect, frequently collaborates with Ethan, a software developer, on different projects. They know each other by name and occasionally engage in small talk. Despite their differing personalities and decision-making styles, they respect each other's professional skills and contributions.")


## Agents Profile Serialization

In [6]:
pks = AgentProfile.all_pks()
pks = list(pks)
res = []
for pk in pks:
    try:
        res.append(AgentProfile.get(pk=pk))
    except:
        print("error")
        pass
pprint(res[0])

AgentProfile(pk='01H5TNE5PQ00AJVSSVB9V2VA9K', first_name='Eli', last_name='Dawson', age=52, occupation='Forensic psychiatrist', gender='Man', gender_pronoun='He/him', public_info='Eli Dawson, a forensic psychiatrist, is a history enthusiast and enjoys collecting vintage books.', big_five='Openness to Experience - High; Conscientiousness - High; Extraversion - Low; Agreeableness - Low; Neuroticism - Low', moral_values=['Liberty'], schwartz_personal_values=['Hedonism'], personality_and_values='Eli Dawson is expressive, assertive, and markedly erratic. Elliott values liberty and enjoyment. His decision-making is strategic, examining the wider context before making decisions.', decision_making_style='Strategic', secret='He secretly funds a college student', model_id='', mbti='INTJ')


In [7]:
agentprofiles_to_csv(res, "../data/sotopia_agentprofiles_v1.csv")
agentprofiles_to_jsonl(res, "../data/sotopia_agentprofiles_v1.jsonl")
agent_profiles = jsonl_to_agentprofiles("../data/sotopia_agentprofiles_v1.jsonl")
pprint(agent_profiles[0])

AgentProfile(pk='01H5TNE5PQ00AJVSSVB9V2VA9K', first_name='Eli', last_name='Dawson', age=52, occupation='Forensic psychiatrist', gender='Man', gender_pronoun='He/him', public_info='Eli Dawson, a forensic psychiatrist, is a history enthusiast and enjoys collecting vintage books.', big_five='Openness to Experience - High; Conscientiousness - High; Extraversion - Low; Agreeableness - Low; Neuroticism - Low', moral_values=['Liberty'], schwartz_personal_values=['Hedonism'], personality_and_values='Eli Dawson is expressive, assertive, and markedly erratic. Elliott values liberty and enjoyment. His decision-making is strategic, examining the wider context before making decisions.', decision_making_style='Strategic', secret='He secretly funds a college student', model_id='', mbti='INTJ')


## Environment Profile Serialization

In [8]:
pks = EnvironmentProfile.all_pks()
pks = list(pks)
res = []
for pk in pks:
    try:
        res.append(EnvironmentProfile.get(pk=pk))
    except:
        print("error")
        pass
pprint(res[0])

EnvironmentProfile(pk='01H7VFHN6NYWSTWCZJE2DCQKTD', codename='facetime_etiquettes', source='social_chemistry', scenario='A couple having a conversation over FaceTime', agent_goals=['Express feelings about the partner taking texts while on FaceTime (<extra_info>Extra information: You find it disrespectful and it makes you feel unimportant.</extra_info>)', "Justify the act of texting while on FaceTime without hurting the partner's feelings (<extra_info>Extra information: You have some urgent and important messages to respond to.</extra_info>)"], relationship=<RelationshipType.romantic_relationship: 4>, age_constraint='[(18, 70), (18, 70)]', occupation_constraint='nan', agent_constraint=None)


In [9]:
environmentprofiles_to_csv(res, "../data/sotopia_environmentprofiles_v1.csv")
environmentprofiles_to_jsonl(res, "../data/sotopia_environmentprofiles_v1.jsonl")
environment_profiles = jsonl_to_environmentprofiles(
    "../data/sotopia_environmentprofiles_v1.jsonl"
)
pprint(environment_profiles[0])

EnvironmentProfile(pk='01H7VFHN6NYWSTWCZJE2DCQKTD', codename='facetime_etiquettes', source='social_chemistry', scenario='A couple having a conversation over FaceTime', agent_goals=['Express feelings about the partner taking texts while on FaceTime (<extra_info>Extra information: You find it disrespectful and it makes you feel unimportant.</extra_info>)', "Justify the act of texting while on FaceTime without hurting the partner's feelings (<extra_info>Extra information: You have some urgent and important messages to respond to.</extra_info>)"], relationship=<RelationshipType.romantic_relationship: 4>, age_constraint='[(18, 70), (18, 70)]', occupation_constraint='nan', agent_constraint=None)


## EnvAgentComboStorage

In [10]:
pks = EnvAgentComboStorage.all_pks()
pks = list(pks)
res = []
for pk in pks:
    try:
        res.append(EnvAgentComboStorage.get(pk=pk))
    except:
        print("error")
        pass
pprint(res[0])

EnvAgentComboStorage(pk='01H7ZJQMW2YQEBDXRETBV48NQ6', env_id='01H7VFHPDE1AM74JSR8KBJJF3A', agent_ids=['01H5TNE5PKW8P500417PMSGSAC', '01H5TNE5PPK39HR52G61PQ5KQ7'])


In [11]:
envagnetcombostorage_to_csv(res, "../data/sotopia_envagnetcombostorage_v1.csv")
envagnetcombostorage_to_jsonl(res, "../data/sotopia_envagnetcombostorage_v1.jsonl")
envagnetcombostorage = jsonl_to_envagnetcombostorage(
    "../data/sotopia_envagnetcombostorage_v1.jsonl"
)
pprint(envagnetcombostorage[0])

EnvAgentComboStorage(pk='01H7ZJQMW2YQEBDXRETBV48NQ6', env_id='01H7VFHPDE1AM74JSR8KBJJF3A', agent_ids=['01H5TNE5PKW8P500417PMSGSAC', '01H5TNE5PPK39HR52G61PQ5KQ7'])
